# 实验对比：自定义框架 vs. 默认 LC2013

[cite_start]本 Notebook 用于运行 A/B 对比测试，评估论文中提出的自定义换道框架 [cite: 257-376][cite_start]（实验 B）与 SUMO 的默认 LC2013 [cite: 161-173] 换道模型（实验 A）之间的性能差异。

**对比指标:**
1.  **吞吐量 (Throughput)**
2.  **平均速度 (Average Speed)**
3.  [cite_start]**2D-TTC 安全性 (2D-TTC)** [cite: 307-313]

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import traci
import traci.constants as tc

# =================================================================
# 1. 基础配置与导入
# =================================================================
# 路径设置：直接定位项目根目录
CURRENT_DIR = os.path.abspath(os.getcwd())
PROJECT_ROOT = os.path.dirname(CURRENT_DIR) if "controller" not in os.listdir(CURRENT_DIR) else CURRENT_DIR

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# 直接导入核心模块（不再试错，如果缺少模块直接报错提醒）
from controller.config import CONFIG
from controller.controllers.lane_manager import LaneManager
from controller.controllers.vehicle_controller import VehicleController, MODE_NO_CONTROL, MODE_BASELINE, MODE_CUSTOM
from controller.environment.twod_ttc_calculator import calculate_2d_ttc

# =================================================================
# 4. 全局配置
# =================================================================
SCENARIO_PATH = os.path.join(PROJECT_ROOT, "scenario")
SUMO_CONFIG_FILE = os.path.join(SCENARIO_PATH, "test.sumocfg")

CONTROL_EDGES = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]
CONTROL_EDGES_SET = set(CONTROL_EDGES) 

CONTROLLED_LANE_INDEX = 0
FIXED_POLICY_M = 8  
FIXED_POLICY_N = 2  
CRITICAL_TTC_THRESHOLD = 3.0 
MAX_SIMULATION_STEPS = 18000
DATA_RECORD_FREQUENCY = 100 

# =================================================================
# 5. 高性能 TTC 计算器
# =================================================================
class FastTTCCalculator:
    def __init__(self, vehicle_length=5.0, vehicle_width=2.0):
        self.L = vehicle_length
        self.W = vehicle_width

    def calculate_global_metrics(self):
        """
        利用订阅数据和空间过滤进行极速计算
        """
        # getAllSubscriptionResults 在 Libsumo 中是极快的内存读取
        raw_results = traci.vehicle.getAllSubscriptionResults()
        
        states = {}
        target_veh_ids = []

        # --- 第一阶段：解析数据并进行【空间过滤】 ---
        for vid, data in raw_results.items():
            if not data: continue
            
            # 过滤1：完整性检查
            if tc.VAR_ROAD_ID not in data or tc.VAR_POSITION not in data:
                continue

            edge_id = data[tc.VAR_ROAD_ID]
            
            # 过滤2：空间筛选
            if edge_id not in CONTROL_EDGES_SET:
                continue

            pos = data[tc.VAR_POSITION]
            speed = data[tc.VAR_SPEED]
            angle = data[tc.VAR_ANGLE]
            angle_rad = np.radians((90.0 - angle) % 360.0)
            
            states[vid] = (pos[0], pos[1], speed, angle_rad, self.L, self.W)
            target_veh_ids.append(vid)

        if len(target_veh_ids) < 2:
            return np.nan, 0

        ttc_values = []
        critical_count = 0

        # --- 第二阶段：计算 TTC ---
        for vid in target_veh_ids:
            # getLeader 在 Libsumo 下也是内存操作，无需担心开销
            leader_info = traci.vehicle.getLeader(vid, dist=150.0) 
            
            if leader_info:
                leader_id, dist = leader_info
                
                if leader_id in states:
                    state_i = states[vid]
                    state_j = states[leader_id]
                    
                    # 优化：1D TTC 预筛选
                    v_i = state_i[2]
                    v_j = state_j[2]
                    dv = v_i - v_j
                    
                    if dv <= 0.1: 
                        ttc_values.append(1.0)
                        continue
                    
                    simple_ttc = dist / dv
                    
                    if simple_ttc < 10.0:
                        try:
                            ttc = calculate_2d_ttc(state_i, state_j)
                        except:
                            ttc = simple_ttc
                    else:
                        ttc = simple_ttc

                    if 0 < ttc < CRITICAL_TTC_THRESHOLD:
                        ttc_values.append(ttc / CRITICAL_TTC_THRESHOLD)
                        critical_count += 1
                    else:
                        ttc_values.append(1.0)
                else:
                    ttc_values.append(1.0)
            else:
                ttc_values.append(1.0)

        avg_ttc = np.mean(ttc_values) if ttc_values else np.nan
        return avg_ttc, critical_count

# =================================================================
# 6. 仿真主函数
# =================================================================
def run_simulation(control_mode: int, label: str):
    print(f"🚀 [Start] {label}...")
    
    # 确保端口释放
    try: traci.close()
    except: pass

    # 启动 SUMO (Socket模式，最稳)
    sumo_cmd = ["sumo", "-c", SUMO_CONFIG_FILE, "--step-length", "0.1", "--no-warnings", "true", "--no-step-log", "true", "--seed", "9497"]
    if sys.platform.startswith("win"): sumo_cmd[0] += ".exe"
    
    traci.start(sumo_cmd, label="default")

    # 初始化模块
    lane_manager = LaneManager(CONTROL_EDGES, CONTROLLED_LANE_INDEX)
    veh_controller = VehicleController(CONTROL_EDGES)
    ttc_calc = FastTTCCalculator()

    # 设置初始策略
    if control_mode == MODE_NO_CONTROL:
        lane_manager.reset_to_mixed_traffic()
    else:
        lane_manager.apply_lane_strategy(FIXED_POLICY_M, FIXED_POLICY_N)


    results = []
    step = 0
    sum_throughput = 0
    
    try:
        # 循环条件
        while traci.simulation.getMinExpectedNumber() > 0 and step <= MAX_SIMULATION_STEPS:
            
            traci.simulationStep()
            
            # --- 自动订阅 (一次性操作) ---
            departed = traci.simulation.getDepartedIDList()
            if departed:
                for vid in departed:
                    traci.vehicle.subscribe(vid, [
                        tc.VAR_POSITION, 
                        tc.VAR_SPEED, 
                        tc.VAR_ANGLE,
                        tc.VAR_ROAD_ID 
                    ])

            # --- 控制逻辑 ---
            if control_mode != MODE_NO_CONTROL:
                lane_manager.step()
            
            if control_mode == MODE_CUSTOM:
                veh_controller.update_vehicle_states(
                    lane_manager.get_active_hml_lanes(),
                    lane_manager.get_active_cdl_lanes(),
                    lane_manager.get_cdl_start_edge(),
                    control_mode=MODE_CUSTOM
                )
            elif control_mode == MODE_NO_CONTROL:
                # Mode 0: 禁用 SUMO 自主换道，避免外侧车道车辆绕行导致排队被稀释
                veh_controller.enforce_keep_lane_mode()

            # --- 数据记录 ---
            sum_throughput += traci.simulation.getArrivedNumber()

            if step % DATA_RECORD_FREQUENCY == 0:
                # 获取全量订阅数据
                subs = traci.vehicle.getAllSubscriptionResults()
                speeds = [d[tc.VAR_SPEED] for d in subs.values() if tc.VAR_SPEED in d]
                mean_speed = np.mean(speeds) if speeds else np.nan
                
                # 计算 TTC
                avg_ttc, crit_count = ttc_calc.calculate_global_metrics()
                
                results.append({
                    "step": step,
                    "time": traci.simulation.getTime(),
                    "mode": label,
                    "throughput": sum_throughput,
                    "mean_speed": mean_speed,
                    "avg_ttc": avg_ttc,
                    "crit_events": crit_count
                })
            
            step += 1
            
    except Exception as e:
        print(f"  [错误] 仿真中断: {e}")
        import traceback
        traceback.print_exc()
    finally:
        try:
            traci.close()
        except:
            pass
        
    print(f"  [结束] {label} 完成，共 {step} 步。")
    return pd.DataFrame(results)

# =================================================================
# 7. 主执行入口
# =================================================================
if __name__ == "__main__":
    
    if not os.path.exists(SUMO_CONFIG_FILE):
        print(f"错误: 找不到配置文件 {SUMO_CONFIG_FILE}")
    else:
        # 依次运行三种模式
        # 注意：Libsumo 模式下，必须确保每次 run_simulation 彻底 close 后再 start
        
        df_no_control = run_simulation(MODE_NO_CONTROL, "No_Control")
        df_baseline = run_simulation(MODE_BASELINE, "Baseline_LC2013")
        df_custom = run_simulation(MODE_CUSTOM, "Custom_Framework")
        
        # 结果汇总与保存
        dfs = [df_no_control, df_baseline, df_custom]
        if any(not df.empty for df in dfs):
            
            df_no_control.to_csv("result_no_control.csv", index=False)
            df_baseline.to_csv("result_baseline.csv", index=False)
            df_custom.to_csv("result_custom.csv", index=False)
            
            print("\n" + "="*80)
            print("SIMULATION RESULTS SUMMARY")
            print("="*80)
            print(f"{'Mode':<18} | {'Steps':<8} | {'Total Thr':<12} | {'Avg Spd':<10} | {'Avg TTC':<10}")
            print("-" * 80)
            
            metrics = {
                "No Control": df_no_control,
                "Baseline": df_baseline,
                "Custom": df_custom
            }
            
            for name, df in metrics.items():
                if df.empty:
                    print(f"{name:<18} | {'FAILED':<40}")
                    continue
                
                total_thr = df['throughput'].values[-1] if len(df) > 0 else 0
                avg_spd = df['mean_speed'].mean()
                avg_ttc = df['avg_ttc'].mean()
                steps_run = df['step'].iloc[-1] if len(df) > 0 else 0
                
                print(f"{name:<18} | {steps_run:<8} | {total_thr:<12.0f} | {avg_spd:<10.2f} | {avg_ttc:<10.6f}")
            print("-" * 80)
            print("详细数据已保存至当前目录 (.csv)。")
        else:
            print("所有仿真均未产生有效数据。")

🚀 [Start] No_Control...
LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
  [结束] No_Control 完成，共 18001 步。
🚀 [Start] Baseline_LC2013...
LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
  [结束] Baseline_LC2013 完成，共 18001 步。
🚀 [Start] Custom_Framework...
LaneManager (V-Paper) 初始化完毕，管理 10 个 SCL 路段。
VehicleController: 正在缓存 vType 参数...
  [结束] Custom_Framework 完成，共 18001 步。

SIMULATION RESULTS SUMMARY
Mode               | Steps    | Total Thr    | Avg Spd    | Avg TTC   
--------------------------------------------------------------------------------
No Control         | 18000    | 1474         | 11.32      | 1.000000  
Baseline           | 18000    | 2012         | 21.02      | 1.000000  
Custom             | 18000    | 1403         | 11.92      | 0.999938  
--------------------------------------------------------------------------------
详细数据已保存至当前目录 (.csv)。


# 结果分析与可视化

In [ ]:
def summarize_results(df: pd.DataFrame, name: str):
    if df.empty:
        print(f"--- {name} 结果 (无数据) ---")
        return
        
    print(f"\n--- {name} 结果汇总 ---")
    
    # 1. 吞吐量
    total_throughput = df['throughput_veh'].sum()
    print(f"  总吞吐量 (辆): {total_throughput}")
    
    # 2. 平均速度 (m/s -> km/h)
    avg_speed_kph = (df['mean_speed_mps'].mean() * 3.6) # .mean() 自动忽略 nans
    print(f"  平均速度 (km/h): {avg_speed_kph:.2f}")
    
    # 3. 2D-TTC
    total_critical_events = df['critical_ttc_events'].sum()
    print(f"  危险 2D-TTC 事件总数 (TTC < {CRITICAL_TTC_THRESHOLD}s): {total_critical_events}")

# 打印两个实验的汇总
summarize_results(df_baseline, "A: 默认 LC2013 ")
summarize_results(df_custom, "B: 自定义框架 ")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(12, 6))
    
    # 计算累计吞吐量
    df_baseline['cumulative_throughput'] = df_baseline['throughput_veh'].cumsum()
    df_custom['cumulative_throughput'] = df_custom['throughput_veh'].cumsum()
    
    plt.plot(df_baseline['time_s'], df_baseline['cumulative_throughput'], label='默认 LC2013 ')
    plt.plot(df_custom['time_s'], df_custom['cumulative_throughput'], label='自定义框架 ', linestyle='--')
    
    plt.title('对比 1: 累计吞吐量', fontsize=16)
    plt.xlabel('仿真时间 (秒)')
    plt.ylabel('累计吞吐量 (辆)')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("数据不完整，跳过绘制吞吐量图表。")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(12, 6))
    
    # 使用滚动平均 (rolling mean) 使曲线平滑
    ROLLING_WINDOW = 300 # 30秒 (300 步 * 0.1s/步)
    
    baseline_speed_kph = (df_baseline['mean_speed_mps'] * 3.6).rolling(window=ROLLING_WINDOW).mean()
    custom_speed_kph = (df_custom['mean_speed_mps'] * 3.6).rolling(window=ROLLING_WINDOW).mean()
    
    plt.plot(df_baseline['time_s'], baseline_speed_kph, label=f'默认 LC2013  ({ROLLING_WINDOW}步滚动平均)')
    plt.plot(df_custom['time_s'], custom_speed_kph, label=f'自定义框架  ({ROLLING_WINDOW}步滚动平均)', linestyle='--')
    
    plt.title('对比 2: 平均速度', fontsize=16)
    plt.xlabel('仿真时间 (秒)')
    plt.ylabel('平均速度 (km/h)')
    plt.legend()
    plt.grid(True)
    plt.ylim(bottom=0) # 速度不能为负
    plt.show()
else:
    print("数据不完整，跳过绘制速度图表。")

In [ ]:
if not df_baseline.empty and not df_custom.empty:
    plt.figure(figsize=(10, 6))
    
    # 比较两个实验中 *发生* 危险TTC事件的 *总数*
    # (这比平均TTC值更能反映安全性)
    events_baseline = df_baseline['critical_ttc_events'].sum()
    events_custom = df_custom['critical_ttc_events'].sum()
    
    labels = ['默认 LC2013 ', '自定义框架 ']
    values = [events_baseline, events_custom]
    
    plt.bar(labels, values, color=['blue', 'orange'])
    
    plt.title(f'对比 3: 危险 2D-TTC 事件总数 (TTC < {CRITICAL_TTC_THRESHOLD}s)', fontsize=16)
    plt.ylabel('事件总数 (越低越安全)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    for i, v in enumerate(values):
        plt.text(i, v + (max(values) * 0.01), str(v), ha='center', fontweight='bold')
        
    plt.show()
else:
    print("数据不完整，跳过绘制 2D-TTC 图表。")